In [111]:
from elasticsearch import Elasticsearch

es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [112]:
body = {
    "query": {
        "multi_match": {
            "query": "地理内容",
            "fields": ["name", "info"]
        }
    }
}
'''
GET test/_search
`body`
'''
# The multi_match query builds on the match query to allow multi-field queries
# operator, minimum_should_match, fuzziness参数含义参考`match`
es.search(index='test', body=body)

{'took': 6,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 709, 'relation': 'eq'},
  'max_score': 6.3862686,
  'hits': [{'_index': 'test',
    '_type': '_doc',
    '_id': '344',
    '_score': 6.3862686,
    '_source': {'name': '高中地理新课程教学设计与评析',
     'publish': '高等教育出版社',
     'type': '高中通用',
     'author': ['叶回玉', '郑玉清', '陈峰'],
     'info': '《高中地理新课程教学设计与评析》是走进课堂――普通高中新课程案例与评析丛书之一。主要内容是针对高中地理新课程教学内容所设计的教学案例。案例结构包括：教材分析与学情分析、教学目标确定、教学重点和难点解析、教学方法选择、教学过程设计等板块。案例内容涉及：太阳对地球的影响、地球运动、大气运动、锋与天气、气压带和风带、自然界的水循环、海水的运动、水资源利用、山岳的形成、河流地貌、自然地理环境、城市化、城市内部空间结构、农业生产与农业区位选择、家乡的农业园区、工业区位因素与区位选择、地理环境对区域发展的影响、荒漠化的防治、森林的开发和保护、西气东输，以及家乡木雕工艺的发展等主题。作者力图通过31个来自课改一线教师的鲜活教学案例，引发对新课程教学中实际问题的研究和思考。《高中地理新课程教学设计与评析》适合作为高中新课程试验区学科教师培训教材，也适合关注高中课程改革的教研人员阅读参考。',
     'price': 23}},
   {'_index': 'test',
    '_type': '_doc',
    '_id': '526',
    '_score': 6.167991,
    '_source': {'name': '地理信息系统概论（第3版）',
     'publish': '高等教育出版社',
     'type': '信息

In [115]:
es.search(index='test', body={
    "query": {
        "multi_match": {
            "query": "地理内容",
            # The query multiplies the info field’s score by three but leaves the name field’s score unchanged.
            "fields": ["name^5", "info^2"]
        }
    }
})

{'took': 6,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 709, 'relation': 'eq'},
  'max_score': 31.931343,
  'hits': [{'_index': 'test',
    '_type': '_doc',
    '_id': '344',
    '_score': 31.931343,
    '_source': {'name': '高中地理新课程教学设计与评析',
     'publish': '高等教育出版社',
     'type': '高中通用',
     'author': ['叶回玉', '郑玉清', '陈峰'],
     'info': '《高中地理新课程教学设计与评析》是走进课堂――普通高中新课程案例与评析丛书之一。主要内容是针对高中地理新课程教学内容所设计的教学案例。案例结构包括：教材分析与学情分析、教学目标确定、教学重点和难点解析、教学方法选择、教学过程设计等板块。案例内容涉及：太阳对地球的影响、地球运动、大气运动、锋与天气、气压带和风带、自然界的水循环、海水的运动、水资源利用、山岳的形成、河流地貌、自然地理环境、城市化、城市内部空间结构、农业生产与农业区位选择、家乡的农业园区、工业区位因素与区位选择、地理环境对区域发展的影响、荒漠化的防治、森林的开发和保护、西气东输，以及家乡木雕工艺的发展等主题。作者力图通过31个来自课改一线教师的鲜活教学案例，引发对新课程教学中实际问题的研究和思考。《高中地理新课程教学设计与评析》适合作为高中新课程试验区学科教师培训教材，也适合关注高中课程改革的教研人员阅读参考。',
     'price': 23}},
   {'_index': 'test',
    '_type': '_doc',
    '_id': '526',
    '_score': 30.839958,
    '_source': {'name': '地理信息系统概论（第3版）',
     'publish': '高等教育出版社',
     'type': '信

In [114]:
'''
type参数解析:
字段:a, a_score
字段:b, b_score
字段:c, c_score
字段:d, d_score
若:max(a_score, b_score, c_score, d_score) = b_score

best_fields:
    (default) Finds documents which match any field, but uses the _score from the best field.
* 未指定`tie_breaker`:
    b_score
# 未指定`tie_breaker`:
    b_score + tie_breaker * (a_score + c_score + d_score)

most-fields:
    Finds documents which match any field and combines the _score from each field.
    (a_score + b_score + c_score + d_score) / 4

phrase and phrase_prefix:
    the phrase and phrase_prefix types behave just like best_fields, but they use a match_phrase or match_phrase_prefix query instead of a match query.

cross_fields:
    Treats fields with the same analyzer as though they were one big field. Looks for each word in any field.
    The cross_fields type is particularly useful with structured documents where multiple fields should match.
    For instance, when querying the first_name and last_name fields for “Will Smith”, the best match is likely to have “Will” in one field and “Smith” in the other.
    One way of dealing with these types of queries is simply to index the first_name and last_name fields into a single full_name field. Of course, this can only be done at index time.
    The cross_field type tries to solve these problems at query time by taking a term-centric approach. It first analyzes the query string into individual terms, then looks for each term in any of the fields, as though they were one big field.
    The cross_field type can only work in term-centric mode on fields that have the same analyzer.


The best_fields and most_fields types are field-centric—they generate a match query per field.
This means that the operator and minimum_should_match parameters are applied to each field individually, which is probably not what you want.

The fuzziness parameter cannot be used with the phrase or phrase_prefix type.
'''
es.search(index='test', body={
    "query": {
        "multi_match": {
            "query": "地理内容",
            # 类似于`mapping.properties.copy_to`(corss_fields优势:搜索时可修改字段权重);默认,"type": "best_fields"
            "type": "cross_fields",
            "fields": ["name", "info^2"],
            "tie_breaker": 0.3
        }
    }
})


{'took': 5,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 709, 'relation': 'eq'},
  'max_score': 11.260024,
  'hits': [{'_index': 'test',
    '_type': '_doc',
    '_id': '344',
    '_score': 11.260024,
    '_source': {'name': '高中地理新课程教学设计与评析',
     'publish': '高等教育出版社',
     'type': '高中通用',
     'author': ['叶回玉', '郑玉清', '陈峰'],
     'info': '《高中地理新课程教学设计与评析》是走进课堂――普通高中新课程案例与评析丛书之一。主要内容是针对高中地理新课程教学内容所设计的教学案例。案例结构包括：教材分析与学情分析、教学目标确定、教学重点和难点解析、教学方法选择、教学过程设计等板块。案例内容涉及：太阳对地球的影响、地球运动、大气运动、锋与天气、气压带和风带、自然界的水循环、海水的运动、水资源利用、山岳的形成、河流地貌、自然地理环境、城市化、城市内部空间结构、农业生产与农业区位选择、家乡的农业园区、工业区位因素与区位选择、地理环境对区域发展的影响、荒漠化的防治、森林的开发和保护、西气东输，以及家乡木雕工艺的发展等主题。作者力图通过31个来自课改一线教师的鲜活教学案例，引发对新课程教学中实际问题的研究和思考。《高中地理新课程教学设计与评析》适合作为高中新课程试验区学科教师培训教材，也适合关注高中课程改革的教研人员阅读参考。',
     'price': 23}},
   {'_index': 'test',
    '_type': '_doc',
    '_id': '526',
    '_score': 11.042904,
    '_source': {'name': '地理信息系统概论（第3版）',
     'publish': '高等教育出版社',
     'type': '信